# More on the Titanic dataset

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import warnings
warnings.simplefilter('ignore', DeprecationWarning)
warnings.simplefilter('ignore', FutureWarning)

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

def plot_roc_curve(target_test, target_predicted_proba):
    fpr, tpr, thresholds = roc_curve(target_test, target_predicted_proba[:, 1])
    
    roc_auc = auc(fpr, tpr)
    # Plot ROC curve
    plt.plot(fpr, tpr, label='ROC curve (area = %0.3f)' % roc_auc)
    plt.plot([0, 1], [0, 1], 'k--')  # random predictions curve
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.0])
    plt.xlabel('False Positive Rate or (1 - Specifity)')
    plt.ylabel('True Positive Rate or (Sensitivity)')
    plt.title('Receiver Operating Characteristic')
    plt.legend(loc="lower right")

In [ ]:
data = pd.read_csv('./data/titanic_train.csv')

In [ ]:
target = data['Survived'].values

## More feature engineering and richer models

Let us now try to build richer models by including more features as potential predictors for our model.

Categorical variables such as `data.Embarked` or `data.Sex` can be converted as boolean indicators features also known as dummy variables or one-hot-encoded features:

In [ ]:
# I want to change categorical values in numbers

In [ ]:
data.Sex.head(3)

In [ ]:
pd.get_dummies(data.Sex, prefix='Sex').head(5)
# dummy because it's just 0 or 1...if female it's 1
# in terms of machine learning, of course one column is enough

In [ ]:
pd.get_dummies(data.Embarked, prefix='Embarked').head(5)

We can combine those new numerical features with the previous features using `pandas.concat` along `axis=1`:

In [ ]:
rich_features = pd.concat([data.get(['Fare', 'Pclass', 'Age']),
                           pd.get_dummies(data.Sex, prefix='Sex'),
                           pd.get_dummies(data.Embarked, prefix='Embarked')],
                          axis=1)
# axis = 1 means concatenate columns
rich_features.head(5)

By construction the new `Sex_male` feature is redundant with `Sex_female`. Let us drop it:

In [ ]:
rich_features_no_male = rich_features.drop('Sex_male', 1)
rich_features_no_male.head(5)

Let us not forget to impute the median age for passengers without age information:

In [ ]:
rich_features_no_male.count()

In [ ]:
rich_features_final = rich_features_no_male.fillna(rich_features_no_male.dropna().median())
rich_features_final.count()

We can finally cross-validate a logistic regression model on this new data an observe that the mean score has significantly increased:

In [ ]:
%%time

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

logreg = LogisticRegression(C=1.)
scores = cross_val_score(logreg, rich_features_final, target, cv=5, scoring='accuracy')
print("Logistic Regression CV scores:")
print("min: {:.3f}, mean: {:.3f}, max: {:.3f}".format(
    scores.min(), scores.mean(), scores.max()))

**Exercise**:

- change the value of the parameter `C`. Does it have an impact on the score?

- fit a new instance of the logistic regression model on the full dataset.

- plot the weights for the features of this newly fitted logistic regression model.

In [ ]:
%load solutions/04A_plot_logistic_regression_weights.py


# Rich young women like Kate Winslet tend to survive the Titanic better
# than poor men like Leonardo.


### Training Non-linear models: ensembles of randomized trees

`sklearn` also implement non linear models that are known to perform very well for data-science projects where datasets have not too many features (e.g. less than 5000).

In particular let us have a look at Random Forests and Gradient Boosted Trees:

In [ ]:
%%time

from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=100)
scores = cross_val_score(rf, rich_features_final, target, cv=5, n_jobs=4,
                         scoring='accuracy')
print("Random Forest CV scores:")
print("min: {:.3f}, mean: {:.3f}, max: {:.3f}".format(
    scores.min(), scores.mean(), scores.max()))

In [ ]:
%%time

from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1,
                                subsample=.8, max_features=.5)
scores = cross_val_score(gb, rich_features_final, target, cv=5, n_jobs=4,
                         scoring='accuracy')
print("Gradient Boosted Trees CV scores:")
print("min: {:.3f}, mean: {:.3f}, max: {:.3f}".format(
    scores.min(), scores.mean(), scores.max()))

Both models seem to do slightly better than the logistic regression model on this data.

**Exercise**:

- Change the value of the learning_rate and other `GradientBoostingClassifier` parameter, can you get a better mean score?

- Would treating the `PClass` variable as categorical improve the models performance?

- Find out which predictor variables (features) are the most informative for those models.

Hints:

Fitted ensembles of trees have `feature_importances_` attribute that can be used similarly to the `coef_` attribute of linear models.

In [ ]:
%load solutions/04B_more_categorical_variables.py


In [ ]:
%load solutions/04C_feature_importance.py


## Automated parameter tuning

Instead of changing the value of the learning rate manually and re-running the cross-validation, we can find the best values for the parameters automatically (assuming we are ready to wait):

In [ ]:
%%time

from sklearn.model_selection import GridSearchCV
# from sklearn.grid_search.GridSearchCV import GridSearchCV

gb = GradientBoostingClassifier(n_estimators=100, subsample=.8)

params = {
    'learning_rate': [0.05, 0.1, 0.5],
    'max_features': [0.5, 1],
    'max_depth': [3, 4, 5],
}
gs = GridSearchCV(gb, params, cv=5, scoring='roc_auc', n_jobs=4)
gs.fit(rich_features_final, target)

Let us sort the models by mean validation score:

In [ ]:
order=np.argsort(gs.cv_results_['mean_test_score'])[::-1]
for i in range(gs.cv_results_['mean_test_score'].shape[0]):
    print('mean: %f, std: %f, params: %s' % 
          (gs.cv_results_['mean_test_score'][order[i]],
           gs.cv_results_['std_test_score'][order[i]],
           gs.cv_results_['params'][order[i]]))


In [ ]:
gs.best_score_

In [ ]:
gs.best_params_

We should note that the mean scores are very close to one another and almost always within one standard deviation of one another. This means that all those parameters are quite reasonable. The only parameter of importance seems to be the `learning_rate`: 0.5 seems to be a bit too high.

## Avoiding data snooping with pipelines

When doing imputation in pandas, prior to computing the train test split we use data from the test to improve the accuracy of the median value that we impute on the training set. This is actually cheating. To avoid this we should compute the median of the features on the training fold and use that median value to do the imputation both on the training and validation fold for a given CV split.

To do this we can prepare the features as previously but without the imputation: we just replace missing values by the -1 marker value:

In [ ]:
features = pd.concat([data.get(['Fare', 'Age']),
                      pd.get_dummies(data.Sex, prefix='Sex'),
                      pd.get_dummies(data.Pclass, prefix='Pclass'),
                      pd.get_dummies(data.Embarked, prefix='Embarked')],
                     axis=1)
features = features.drop('Sex_male', 1)

# Because of the following bug we cannot use NaN as the missing
# value marker, use a negative value as marker instead:
# https://github.com/scikit-learn/scikit-learn/issues/3044
features = features.fillna(-1)
features.head(5)

We can now use the `Imputer` transformer of scikit-learn to find the median value on the training set and apply it on missing values of both the training set and the test set.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(features.values, target, random_state=0)

In [ ]:
from sklearn.preprocessing import Imputer

imputer = Imputer(strategy='median', missing_values=-1)

imputer.fit(X_train)

The median age computed on the training set is stored in the `statistics_` attribute.

In [ ]:
imputer.statistics_

Imputation can now happen by calling  the transform method:

In [ ]:
X_train_imputed = imputer.transform(X_train)
X_test_imputed = imputer.transform(X_test)

In [ ]:
np.any(X_train == -1)

In [ ]:
np.any(X_train_imputed == -1)

In [ ]:
np.any(X_test == -1)

In [ ]:
np.any(X_test_imputed == -1)

We can now use a pipeline that wraps an imputer transformer and the classifier itself:

In [ ]:
from sklearn.pipeline import Pipeline

imputer = Imputer(strategy='median', missing_values=-1)

classifier = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1,
                                        subsample=.8, max_features=.5)

pipeline = Pipeline([
    ('imp', imputer),
    ('clf', classifier),
])

scores = cross_val_score(pipeline, features.values, target, cv=5, n_jobs=4,
                         scoring='accuracy', )
print(scores.min(), scores.mean(), scores.max())

The mean cross-validation is slightly lower than we used the imputation on the whole data as we did earlier although not by much. This means that in this case the data-snooping was not really helping the model cheat by much.

Let us re-run the grid search, this time on the pipeline. Note that thanks to the pipeline structure we can optimize the interaction of the imputation method with the parameters of the downstream classifier without cheating:

In [ ]:
%%time

params = {
    'imp__strategy': ['mean', 'median'],
    'clf__max_features': [0.5, 1],
    'clf__max_depth': [3, 4, 5],
}
gs = GridSearchCV(pipeline, params, cv=5, scoring='roc_auc', n_jobs=4)
gs.fit(X_train, y_train)

In [ ]:
order=np.argsort(gs.cv_results_['mean_test_score'])[::-1]
for i in range(gs.cv_results_['mean_test_score'].shape[0]):
    print('mean: %f, std: %f, params: %s' % 
          (gs.cv_results_['mean_test_score'][order[i]],
           gs.cv_results_['std_test_score'][order[i]],
           gs.cv_results_['params'][order[i]]))

In [ ]:
gs.best_score_

In [ ]:
plot_roc_curve(y_test, gs.predict_proba(X_test))

In [ ]:
gs.best_params_

# Example from Image Processing

Here we'll take a look at a simple facial recognition example.
Ideally, we would use a dataset consisting of a
subset of the [Labeled Faces in the Wild](http://vis-www.cs.umass.edu/lfw/)
data that is available within scikit-learn with the 'datasets.fetch_lfw_people' function. However, this is a relatively large download (~200MB) so we will do the tutorial on a simpler, less rich dataset. Feel free to explore the LFW dataset at home.

In [ ]:
# from sklearn import datasets
# faces = datasets.fetch_olivetti_faces()
# faces.data.shape

# np.save('olivetti_data.npy',faces['data'])
# np.save('olivetti_images.npy',faces['images'])
# np.save('olivetti_target.npy',faces['target'])

In [ ]:
faces={}
faces['data']= np.load('./data/olivetti_data.npy')
faces['images']= np.load('./data/olivetti_images.npy')
faces['target']= np.load('./data/olivetti_target.npy')

Let's visualize these faces to see what we're working with:

In [ ]:
fig = plt.figure(figsize=(8, 6))
# plot several images
for i in range(15):
    ax = fig.add_subplot(3, 5, i + 1, xticks=[], yticks=[])
#     ax.imshow(faces.images[i], cmap=plt.cm.bone)
    ax.imshow(faces['images'][i], cmap=plt.cm.bone)

One thing to note is that these faces have already been localized and scaled
to a common size.  This is an important preprocessing piece for facial
recognition, and is a process that can require a large collection of training
data.  This can be done in scikit-learn, but the challenge is gathering a
sufficient amount of training data for the algorithm to work

Fortunately, this piece is common enough that it has been done.  One good
resource is [OpenCV](https://docs.opencv.org/2.4/modules/contrib/doc/facerec/facerec_tutorial.html), the
*Open Computer Vision Library*.

We'll perform a Support Vector classification of the images.  We'll
do a typical train-test split on the images to make this happen:

In [ ]:
from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(faces.data,
#         faces.target, random_state=0)
X_train, X_test, y_train, y_test = train_test_split(faces['data'],
        faces['target'], random_state=0)

print(X_train.shape, X_test.shape)

## Preprocessing: Principal Component Analysis

4096 dimensions is a lot for SVM.  We can use PCA to reduce these 4096 features to a manageable
size, while maintaining most of the information in the dataset.  Here it is useful to use a variant
of PCA called ``RandomizedPCA``, which is an approximation of PCA that can be much faster for large
datasets.  The interface is the same as the normal PCA we saw earlier:

In [ ]:
from sklearn import decomposition
pca = decomposition.PCA(svd_solver='randomized',n_components=150, whiten=True)
pca.fit(X_train)

One interesting part of PCA is that it computes the "mean" face, which can be
interesting to examine:

In [ ]:
# plt.imshow(pca.mean_.reshape(faces.images[0].shape),
#            cmap=plt.cm.bone)

plt.imshow(pca.mean_.reshape(faces['images'][0].shape),
           cmap=plt.cm.bone)

The principal components measure deviations about this mean along orthogonal axes.
It is also interesting to visualize these principal components:

In [ ]:
print(pca.components_.shape)

In [ ]:
fig = plt.figure(figsize=(16, 6))
for i in range(30):
    ax = fig.add_subplot(3, 10, i + 1, xticks=[], yticks=[])
#     ax.imshow(pca.components_[i].reshape(faces.images[0].shape), cmap=plt.cm.bone)
    ax.imshow(pca.components_[i].reshape(faces['images'][0].shape), cmap=plt.cm.bone)

The components ("eigenfaces") are ordered by their importance from top-left to bottom-right.
We see that the first few components seem to primarily take care of lighting
conditions; the remaining components pull out certain identifying features:
the nose, eyes, eyebrows, etc.

With this projection computed, we can now project our original training
and test data onto the PCA basis:

In [ ]:
X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)

In [ ]:
print(X_train_pca.shape)
print(X_test_pca.shape)

These projected components correspond to factors in a linear combination of
component images such that the combination approaches the original face.

## Doing the Learning: Support Vector Machines

Now we'll perform support-vector-machine classification on this reduced dataset:

In [ ]:
from sklearn import svm
clf = svm.SVC(C=5., gamma=0.001)
clf.fit(X_train_pca, y_train)

Finally, we can evaluate how well this classification did.  First, we might plot a
few of the test-cases with the labels learned from the training set:

In [ ]:
X_test_pca[0][np.newaxis,:].shape

In [ ]:
fig = plt.figure(figsize=(8, 6))
for i in range(15):
    ax = fig.add_subplot(3, 5, i + 1, xticks=[], yticks=[])
#     ax.imshow(X_test[i].reshape(faces.images[0].shape),
#               cmap=plt.cm.bone)
    ax.imshow(X_test[i].reshape(faces['images'][0].shape),
              cmap=plt.cm.bone)
    y_pred = clf.predict(X_test_pca[i][np.newaxis,:])[0]
    color = ('black' if y_pred == y_test[i]
             else 'red')
    ax.set_title(y_pred, fontsize='small', color=color)

The classifier is correct on an impressive number of images given the simplicity
of its learning model!  Using a linear classifier on 150 features derived from
the pixel-level data, the algorithm correctly identifies a large number of the
people in the images.

Again, we can
quantify this effectiveness using one of several measures from the ``sklearn.metrics``
module.  First we can do the classification report, which shows the precision,
recall and other measures of the "goodness" of the classification:

In [ ]:
from sklearn import metrics
y_pred = clf.predict(X_test_pca)
print(metrics.classification_report(y_test, y_pred))

Another interesting metric is the *confusion matrix*, which indicates how often
any two items are mixed-up.  The confusion matrix of a perfect classifier
would only have nonzero entries on the diagonal, with zeros on the off-diagonal.

In [ ]:
print(metrics.confusion_matrix(y_test, y_pred))

# Quick Note

Here we have used PCA "eigenfaces" as a pre-processing step for facial recognition.
The reason we chose this is because PCA is a broadly-applicable technique, which can
be useful for a wide array of data types.  Research in the field of facial recognition
in particular, however, has shown that other more specific feature extraction methods
are can be much more effective.

# Credits

Thanks to:

- Again [Bartosz Teleńczuk](https://datascience.telenczuk.pl/teaching/) and [Software Carpentry team](https://software-carpentry.org/) for providing notebook from which these are inspired

- Kaggle for setting up the Titanic challenge.

- This blog post by Philippe Adjiman for inspiration:

http://www.philippeadjiman.com/blog/2013/09/12/a-data-science-exploration-from-the-titanic-in-r/